In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

plt.rc('font', family='Malgun Gothic')  # 폰트 지정
plt.rc('axes', unicode_minus=False)  # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'  # 그래프 글씨 뚜렷

In [2]:
df_train = pd.read_csv("../data/df_train_004.csv")
df_test = pd.read_csv("../data/df_test_004.csv")

df_train['target'] = df_train['사망자수']*10 + df_train['중상자수'] * \
    5 + df_train['경상자수']*3 + df_train['부상자수']
df_train.drop(columns=['사망자수', '중상자수', '경상자수', '부상자수'], axis=1, inplace=True)

In [3]:
X = df_train.drop(columns='target', axis=1)
X = X + 1
X = np.log1p(X)
y = df_train['target']

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# gradientboost

In [9]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_log_error

In [22]:
# Gradient Boosting 회귀 모델 초기화 및 학습
model = GradientBoostingRegressor(loss = 'huber',n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(x_train, y_train)

GradientBoostingRegressor(loss='huber', random_state=42)

In [23]:
# 예측
y_pred = model.predict(x_test)

In [24]:
# MSLE(Mean Squared Logarithmic Error) 계산
msle = mean_squared_log_error(y_test, y_pred)
print(f'MSLE: {msle}')

MSLE: 0.19788904252911013


In [25]:
sub_file = pd.read_csv("../data/sample_submission.csv")

In [26]:
sub_file['ECLO'] = np.round(model.predict(df_test))

In [27]:
sub_file.ECLO.value_counts()

5.0    5069
4.0    4770
3.0    1108
6.0      14
8.0       1
7.0       1
Name: ECLO, dtype: int64

In [28]:
sub_file.to_csv("../data/sub_file/015_gradientboost.csv", index=False)